In [2]:
from configparser import ConfigParser

# Best format for the result of the walk function in PNG
from IPython.display import display, Image

import pandas as pd

from psycopg2 import connect
from pygwalker import walk

In [3]:
def format_cursor(cursor):
    return [cursor.fetchall(),
             (desc[0] for desc in cursor.description)]

In [4]:
def data(data_in_query):
    return pd.DataFrame(data_in_query[0],
                        columns=data_in_query[1])

In [5]:
config = ConfigParser()
config.read('TEST.ini', encoding="utf-8")

config = {i: config['postgresql'][i]
           for i in config['postgresql']}

## ПРИРОСТ ЧИСЛА ЛЮДЕЙ НА ПЛОЩАДКЕ

In [6]:
with connect(**config) as connect:
    with connect.cursor() as cursor_Relative_dynamics_people:
        cursor_Relative_dynamics_people.execute("""
SELECT date AS date,
new_users,
new_couriers,
total_couriers,
total_users,
ROUND((new_users::DECIMAL / LAG(new_users, 1) OVER (ORDER BY date) - 1) * 100, 2)::FLOAT new_users_change,
ROUND((new_couriers::DECIMAL / LAG(new_couriers, 1) OVER (ORDER BY date) - 1) * 100, 2)::FLOAT new_couriers_change,
ROUND((total_users::DECIMAL / LAG(total_users, 1) OVER (ORDER BY date) - 1) * 100, 2)::FLOAT total_users_growth,
ROUND((total_couriers::DECIMAL / LAG(total_couriers, 1) OVER (ORDER BY date) - 1) * 100, 2)::FLOAT total_couriers_growth
FROM (
    SELECT COUNT(DISTINCT user_id) new_users,
    (SUM(COUNT(DISTINCT user_id)) OVER (ORDER BY date))::INTEGER total_users,
    date
    FROM (
        SELECT DISTINCT user_id,
        date_trunc('day', MIN(time) OVER (PARTITION BY user_id ORDER BY order_id)) AS date
        FROM user_actions) t1
    GROUP BY date
    ORDER BY date
) users
JOIN (
    SELECT COUNT(DISTINCT courier_id) new_couriers,
    (SUM(COUNT(DISTINCT courier_id)) OVER (ORDER BY date))::INTEGER total_couriers,
    date
    FROM (
        SELECT DISTINCT courier_id,
            date_trunc('day', MIN(time) OVER (PARTITION BY courier_id ORDER BY courier_id)) AS date
            FROM courier_actions
        ) t1
    GROUP BY date
    ORDER BY date
) couriers
USING(date)
ORDER BY date""")
        df_cursor_Relative_dynamics_people = data(
            format_cursor(cursor_Relative_dynamics_people))

print(df_cursor_Relative_dynamics_people, sep="\n \n")

         date  new_users  new_couriers  total_couriers  total_users  \
0  2022-08-24        134            95              95          134   
1  2022-08-25        829           242             337          963   
2  2022-08-26       1017           219             556         1980   
3  2022-08-27       1225           186             742         3205   
4  2022-08-28       1487           213             955         4692   
5  2022-08-29       1309           109            1064         6001   
6  2022-08-30       1204           127            1191         7205   
7  2022-08-31       1420           195            1386         8625   
8  2022-09-01       1513           205            1591        10138   
9  2022-09-02       1887           229            1820        12025   
10 2022-09-03       1924           161            1981        13949   
11 2022-09-04       1952           205            2186        15901   
12 2022-09-05       1397           120            2306        17298   
13 202

In [7]:
??walk

Signature:
walk(
    df: Union[~DataFrame, Any],
    gid: Union[int, str] = None,
    *,
    env: Literal['Jupyter', 'Streamlit', 'JupyterWidget'] = 'JupyterWidget',
    fieldSpecs: Optional[Dict[str, pygwalker.data_parsers.base.FieldSpec]] = None,
    hideDataSourceConfig: bool = True,
    themeKey: Literal['vega', 'g2'] = 'g2',
    dark: Literal['media', 'light', 'dark'] = 'media',
    return_html: bool = False,
    spec: str = '',
    use_preview: bool = True,
    store_chart_data: bool = False,
    use_kernel_calc: bool = False,
    **kwargs,
)
Source:   
def walk(
    df: Union[DataFrame, Any],
    gid: Union[int, str] = None,
    *,
    env: Literal['Jupyter', 'Streamlit', 'JupyterWidget'] = 'JupyterWidget',
    fieldSpecs: Optional[Dict[str, FieldSpec]] = None,
    hideDataSourceConfig: bool = True,
    themeKey: Literal['vega', 'g2'] = 'g2',
    dark: Literal['media', 'light', 'dark'] = 'media',
    return_html: bool = False,
    spec: str = "",
    use_preview: bool = True,
  

In [ ]:
walk(df_cursor_Relative_dynamics_people,
     spec='config_pygwalker.json', dark='dark')

In [57]:
# display(
#     Image(filename=r'results_pygwalker\new_total_peoples\new_peoples_change_1.png'),
#     Image(filename=r'results_pygwalker\new_total_peoples\new_peoples_change_2.png'),
#     Image(filename=r'results_pygwalker\new_total_peoples\total_peoples_growth_1.png'),
#     Image(filename=r'results_pygwalker\new_total_peoples\total_peoples_growth_2.png')
# )